# Lab 2

## Import libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat
import math
import scipy.stats

In [37]:
mu = 67.21
sigma_2 = 248.21
gamma = 0.95

## Read data

In [2]:
df = pd.read_excel('/Users/dkravchenko/Jupyter_Notebookes/MathStat/lab2/data.xlsx')
df.head(10)

,reading score,writing score
0,47,32
1,71,21
2,73,35
3,86,36
4,100,10
5,86,10
6,75,54
7,52,35
8,48,55
9,71,10


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   reading score  200 non-null    int64
 1   writing score  200 non-null    int64
dtypes: int64(2)
memory usage: 3.2 KB


#### Reading scores serie

In [4]:
reading_score = df['reading score'].sort_values()

In [5]:
reading_score.describe()

count    200.000000
mean      68.590000
std       15.711338
min       32.000000
25%       56.750000
50%       68.000000
75%       80.250000
max      100.000000
Name: reading score, dtype: float64

##### Intervals

In [40]:
#parametrs
n = reading_score.count()
k = int(n ** (1/2))
r = int(reading_score.max() - reading_score.min())
l = r / k

#intervals
a_i = []
a_i_plus = []

for i in range(k):
    a_i.append(reading_score.min() + l * i)
    a_i_plus.append(reading_score.min() + l * (i + 1))
    
reading_interval = pd.DataFrame({'a_i': a_i,
                                 'a_i+1': a_i_plus})
    
reading_interval.head(k)

,a_i,a_i+1
0,32.000000,36.857143
1,36.857143,41.714286
2,41.714286,46.571429
3,46.571429,51.428571
4,51.428571,56.285714
5,56.285714,61.142857
6,61.142857,66.000000
7,66.000000,70.857143
8,70.857143,75.714286
9,75.714286,80.571429


##### Calculating number of values in intervals

In [41]:
intervals_values = []
values = reading_score.value_counts().sort_index()

for i in range(k):
    if ((i+1) % 2) and (i != (k - 1)):
        intervals_values.append(sum(values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i] - 0.01)]))
    else:
        intervals_values.append(sum(values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i])]))

reading_interval['values'] = intervals_values
reading_interval.head(k)

,a_i,a_i+1,values
0,32.000000,36.857143,5
1,36.857143,41.714286,4
2,41.714286,46.571429,5
3,46.571429,51.428571,16
4,51.428571,56.285714,20
5,56.285714,61.142857,14
6,61.142857,66.000000,23
7,66.000000,70.857143,24
8,70.857143,75.714286,18
9,75.714286,80.571429,21


##### Adding parameters

In [42]:
c = []
extra = []
for i in range(k):
    c.append((a_i[i] + a_i_plus[i]) / 2)
    extra.append(intervals_values[i] * ((c[i] - mu) ** 2))
reading_interval['c'] = c
reading_interval['mi*(ci-mu)^2'] = extra
reading_interval.head(k)

,a_i,a_i+1,values,c,mi*(ci-mu)^2
0,32.000000,36.857143,5,34.428571,5373.110296
1,36.857143,41.714286,4,39.285714,3119.062931
2,41.714286,46.571429,5,44.142857,2660.465398
3,46.571429,51.428571,16,49.000000,5305.665600
4,51.428571,56.285714,20,53.857143,3565.975878
5,56.285714,61.142857,14,58.714286,1010.480257
6,61.142857,66.000000,23,63.571429,304.501647
7,66.000000,70.857143,24,68.428571,35.637992
8,70.857143,75.714286,18,73.285714,664.457473
9,75.714286,80.571429,21,78.142857,2510.074671


#### Writting scores series

In [81]:
writting_score = df['writing score'].sort_values()

In [82]:
writting_score.describe()

count    200.000000
mean      25.065000
std       19.954994
min        0.000000
25%       10.000000
50%       19.000000
75%       34.250000
max      104.000000
Name: writing score, dtype: float64

### Reading serie

#### Data evaluation

In [9]:
reading_parameters = {'E': stat.mean(reading_score),
                      'D': stat.variance(reading_score),
                      'Median': stat.median(reading_score),
                      'p_D': stat.pvariance(reading_score),
                      'stdev': stat.stdev(reading_score)}

for key, value in reading_parameters.items():
    print('------------------')
    print('|%-8s %7.2f|' % (key + ':', value))
print('------------------')

------------------
|E:         68.59|
------------------
|D:        246.85|
------------------
|Median:    68.00|
------------------
|p_D:      245.61|
------------------
|stdev:     15.71|
------------------


#### Confidence intervals reading score

In [70]:
#quaniles
quantil1 = scipy.stats.norm.ppf(1 - ((1 - gamma) / 2))
quantil2 = 1.972 #Стьюдент
quantil3 = scipy.stats.chi2.ppf((1 + gamma) / 2, n)
quantil4 = scipy.stats.chi2.ppf((1 + gamma) / 2, n - 1)
quantil5 = scipy.stats.chi2.ppf((1 - gamma) / 2, reading_parameters['E'])
quantil6 = scipy.stats.chi2.ppf((1 - gamma) / 2, reading_parameters['E'] - 1)

print('-------------------------------------------------------------------------------')
print('| %-10s | %-10s | %-10s | %-10s | %-10s | %-10s |' % ('Quantile 1', 'Quantile 2', 'Quantile 3', 'Quantile 4', 'Quantile 5', 'Quantile 6'))
print('-------------------------------------------------------------------------------')
print('|%12.3f|%12.3f|%12.3f|%12.3f|%12.3f|%12.3f|' % (quantil1, quantil2, quantil3, quantil4, quantil5, quantil6))
print('-------------------------------------------------------------------------------')

-------------------------------------------------------------------------------
| Quantile 1 | Quantile 2 | Quantile 3 | Quantile 4 | Quantile 5 | Quantile 6 |
-------------------------------------------------------------------------------
|       1.960|       1.972|     241.058|     239.960|      47.583|      46.751|
-------------------------------------------------------------------------------


##### For 𝜇 with a known 𝜎^2

In [25]:
left_border = reading_parameters['E'] - quantil1 * (sigma_2 / n) ** (1/2) 
rigth_border = reading_parameters['E'] + quantil1 * (sigma_2 / n) ** (1/2) 
print('Interval: (%2.3f, %2.3f)' % (left_border, rigth_border))

Interval: (66.407, 70.773)


##### For 𝜇 with a unknown 𝜎^2

In [26]:
left_border = reading_parameters['E'] - quantil2 * reading_parameters['stdev'] / ((n-1) ** 1/2) 
rigth_border = reading_parameters['E'] + quantil2 * reading_parameters['stdev'] / ((n-1) ** 1/2) 
print('Interval: (%2.3f, %2.3f)' % (left_border, rigth_border))

Interval: (68.279, 68.901)


##### For 𝜎^2 with a unknown 𝜇

In [72]:
left_border = n * reading_parameters['p_D'] / quantil4
rigth_border = n * reading_parameters['p_D'] / quantil6
print('Interval: (%2.3f, %2.3f)' % (left_border, rigth_border))

Interval: (204.711, 1050.721)


##### For 𝜎^2 with a known 𝜇

In [73]:
left_border = sum(reading_interval['mi*(ci-mu)^2']) / quantil3
rigth_border = sum(reading_interval['mi*(ci-mu)^2']) / quantil5
print('Interval: (%2.3f, %2.3f)' % (left_border, rigth_border))

Interval: (202.058, 1023.642)


##### Python interval

In [108]:
interval = scipy.stats.t.interval(alpha=gamma, df=n, loc=reading_parameters['E'], scale=scipy.stats.sem(reading_score))
print('Interval: (%2.3f, %2.3f)' % (interval[0], interval[1]))

Interval: (66.399, 70.781)


### Writting serie

#### Data evaluation

In [80]:
writting_parameters = {'E': stat.mean(writting_score),
                      'D': stat.variance(writting_score),
                      'Median': stat.median(writting_score),
                      'p_D': stat.pvariance(writting_score),
                      'stdev': stat.stdev(writting_score)}

for key, value in writting_parameters.items():
    print('------------------')
    print('|%-8s %7.2f|' % (key + ':', value))
print('------------------')

------------------
|E:         25.07|
------------------
|D:        398.20|
------------------
|Median:    19.00|
------------------
|p_D:      396.21|
------------------
|stdev:     19.95|
------------------


In [84]:
#quantile 
print(quantil1)

1.959963984540054


#### Interval for exponential distribution param

In [88]:
left_border = (1/writting_parameters['E']) - (quantil1 / (writting_parameters['E'] * (n ** (1/2))))
rigth_border = (1/writting_parameters['E']) + (quantil1 / (writting_parameters['E'] * (n ** (1/2))))
print('Interval: (%2.3f, %2.3f)' % (left_border, rigth_border))

Interval: (0.034, 0.045)


#### Interval for Poisson distribution param

In [90]:
left_border = writting_parameters['E'] - quantil1 * ((writting_parameters['E'] / n) ** (1/2))
rigth_border = writting_parameters['E'] + quantil1 * ((writting_parameters['E'] / n) ** (1/2))
print('Interval: (%2.3f, %2.3f)' % (left_border, rigth_border))

Interval: (24.371, 25.759)


## Conclusion

##### Доверительные интервалы полученные различными методами почти совпали, что объснется тем, что известные значения близки к выборочным оценкам.

##### Работу выполнял Кравченко Д.
##### Группа R3338
##### Вариант 3